In [1]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

import numpy as np
from collections import deque
import random
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import pickle
import os
from datetime import datetime

from IPython.display import display, clear_output

# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device='cpu'

In [2]:
def createDir(dir):
    if not os.path.isdir(dir):
        os.mkdir(dir)

In [3]:
class DQN():
    def __init__(
            self,
            env,
            gamma,
            epsilon,
            numEpisodes,
            stateDim,
            actionDim,
            replayBufferSize,
            batchSize,
            epsilonDecay,
            epsilonDecayStart,
            lr,
            checkpointDir="checkpoints",
            checkpointFrequency = 10000,
            runId = datetime.now().strftime("%y_%m_%d__%H_%M_%S"),
    ):
        # Initialize DQN parameters and variables
        self.env = env  # Environment
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Initial exploration rate
        self.numEpisodes = numEpisodes  # Number of training episodes
        self.stateDim = stateDim  # Dimensionality of state space
        self.actionDim = actionDim  # Dimensionality of action space
        self.replayBufferSize = replayBufferSize  # Size of the replay buffer
        self.batchSize = batchSize  # Mini-batch size for training
        self.epsilonDecay = epsilonDecay  # Rate of exploration decay
        self.epsilonDecayStart = epsilonDecayStart  # Episode at which to start epsilon decay
        self.checkpointDir = checkpointDir  # Folder for storing checkpoints
        self.checkpointFrequency = checkpointFrequency  # Frequency (in steps) for storing checkpoints
        self.runId = runId  # A run id to uniquely characterize every run / train attempt. By default takes current time as value
 
        self.writer = SummaryWriter()

        # Initialize counters and lists
        self.steps = 0
        self.episodes = 0
        self.sumRewardsEpisode = []  # Track sum of rewards per episode

        # Initialize replay buffer
        self.replayBuffer = deque(maxlen=self.replayBufferSize)

        # Initialize neural network
        self.network = self.createNetwork(self.stateDim, self.actionDim)

        # Initialize actions history
        self.actionsHistory = []

        # Initialize optimizer
        self.optim = torch.optim.Adam(self.network.parameters(), lr=lr)

    # Define the neural network architecture
    def createNetwork(self, inDim, outDim):
        return nn.Sequential(
            nn.Linear(inDim, 64),  # Input layer with ReLU activation
            nn.ReLU(),
            
            nn.Linear(64, 64),  # Hidden layer with ReLU activation
            nn.ReLU(),
            
            nn.Linear(64, 32),  # Another hidden layer with ReLU activation
            nn.ReLU(),
            
            nn.Linear(32, outDim),  # Output layer
        ).to(device)
    
    # Define custom loss function for DQN
    def my_loss(self, y_pred, y_true):
        # Calculate mean squared error loss only for the actions taken
        s1, s2 = y_true.shape
        loss = F.mse_loss(y_pred[torch.arange(s1), self.actionsHistory], y_true[torch.arange(s1), self.actionsHistory])
        return loss    
    
    # Train the network over multiple episodes
    def trainingEpisodes(self):
        # Initialize lists to store episode rewards and average Q values
        self.sumRewardsEpisode = []
        self.avgQ = []
        # Iterate over episodes
        for episode in tqdm(range(self.numEpisodes)):
            rewards = []
            # Reset environment and get initial state
            (currentState, _) = self.env.reset()
            terminalState = False
            self.Qs = []  # List to store Q values
            # Loop until episode termination
            while not terminalState:
                # Select action using epsilon-greedy policy
                action = self.selectAction(currentState, episode)
                # Take action and observe next state and reward
                (nextState, reward, terminalState, _, _) = self.env.step(action)
                rewards.append(reward)
                # Store experience in replay buffer
                self.replayBuffer.append((currentState, action, reward, nextState, terminalState))
                # Train the network
                self.trainNetwork()
                currentState = nextState
            
            # Store episode reward and average Q value
            self.sumRewardsEpisode.append(np.sum(rewards))
            self.avgQ.append(np.mean(self.Qs))

            # Updating tensor board
            self.writer.add_scalar("steps", self.steps, self.episodes)
            self.writer.add_scalar("rewards", self.sumRewardsEpisode[self.episodes], self.episodes)
            self.writer.add_scalar("avg Q value", self.avgQ[self.episodes], self.episodes)

            self.episodes += 1
    
    # Select action using epsilon-greedy policy
    def selectAction(self, state, index):
        # Initial exploration phase
        if index == 0:
            return np.random.choice(self.actionDim)
        
        rand = np.random.random()
        # Decay exploration rate over time
        if index > self.epsilonDecayStart:
            self.epsilon *= self.epsilonDecay

        if rand < self.epsilon:
            return np.random.choice(self.actionDim)  # Random action
        # Exploit learned policy
        Q = self.network(torch.tensor(state.reshape(1, 4)).to(device))
        return torch.argmax(Q[0]).item()
    
    # Train the network using experiences from replay buffer
    def trainNetwork(self):
        if len(self.replayBuffer) > self.batchSize:
            # Sample a mini-batch from replay buffer
            randomBatch = random.sample(self.replayBuffer, self.batchSize)
            currentStateBatch = torch.zeros(size=(self.batchSize, 4)).to(device)
            nextStateBatch = torch.zeros(size=(self.batchSize, 4)).to(device)

            for index, tupleS in enumerate(randomBatch):
                # Extract data from the mini-batch
                currentStateBatch[index, :] = torch.tensor(tupleS[0])
                nextStateBatch[index, :] = torch.tensor(tupleS[3])

            # Set network to evaluation mode
            self.network.eval()
            QS_ = self.network(nextStateBatch)  # Compute Q-values for next states

            Y = torch.zeros((self.batchSize, 2)).to(device)
            self.actionsHistory = []
            rewards_batch = []
            for idx, (currentState, action, reward, nextState, terminated) in enumerate(randomBatch):
                rewards_batch.append(reward)
                if terminated:
                    y = reward
                else:
                    # Calculate target Q-value using target network
                    y = reward + self.gamma * torch.max(QS_[idx])
                self.actionsHistory.append(action)
                Y[idx, action] = y
            
            # Set network back to training mode
            self.network.train()

            # Zero out gradients
            self.optim.zero_grad()

            # Compute Q-values for current states
            QS = self.network(currentStateBatch).to(device)

            # Compute loss and backpropagate
            loss = self.my_loss(QS, Y)
            loss.backward()
            self.optim.step()  # Update network parameters

            self.Qs.append(QS.flatten().sum().item())  # Track sum of Q-values

            self.steps += 1  # Increment step counter

            # Saving checkpoints periodically
            if self.steps % self.checkpointFrequency == 0:
                self.save_checkpoint()

    # A function to save checkpoints
    def save_checkpoint(self):
        # !TODO self.writer is not picklable. this is a workaround I am not too happy with
        tempWriter = self.writer
        self.writer = None

        createDir(self.checkpointDir)
        createDir(os.path.join(self.checkpointDir, self.runId))
        checkpointPath = os.path.join(self.checkpointDir, self.runId, f"checkpoint-{self.steps}.pkl")
        with open(checkpointPath, "wb") as f:
            pickle.dump(self, f)

        self.writer = tempWriter
    
    # A class method to load a checkpoint
    @classmethod
    def load_checkpoint(cls, checkpointPath):
        with open(checkpointPath, "rb") as f:
            dqn = pickle.load(f)
        return dqn

In [4]:
env = gym.make('CartPole-v1')

In [5]:
dqn = DQN(
    env=env,
    gamma=1,
    epsilon=0.1,
    numEpisodes=1000,
    stateDim=4,
    actionDim=2,
    replayBufferSize=256,
    batchSize=64,
    epsilonDecay=0.99,
    epsilonDecayStart=200,
    lr = 1e-3,
)

In [6]:
dqn.trainingEpisodes()

  0%|          | 0/1000 [00:00<?, ?it/s]

c:\programming\projects\pj_env\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
c:\programming\projects\pj_env\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\programming\projects\pj_env\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


KeyboardInterrupt: 

## Todo
- .py
- params (yml)
- train, experiment
- use image
  - prepare dataset
  - train backbone
  - retrain 

## Observations
- Dropout significantly slows down training and is not worth it